In [1]:
from __future__ import print_function
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def alignImages_ORB(im1, im2):
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create(MAX_MATCHES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = list(matcher.match(descriptors1, descriptors2, None))

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
    print(len(matches))

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    
    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return imMatches, im1Reg, h

In [3]:
def alignImages_SIFT(im1, im2):
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    orb = cv2.SIFT_create(MAX_MATCHES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.BFMatcher()
    matches = matcher.match(descriptors1, descriptors2, k=2)

    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])

    # Draw top matches
    imMatches = cv2.drawMatchesKnn(im1, keypoints1, im2, keypoints2, matches, None)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return imMatches, im1Reg, h

In [4]:
MAX_MATCHES = 10000
GOOD_MATCH_PERCENT = 0.001

In [5]:
refFilename = "ace_heart.png"
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
# imReference = cv2.resize(imReference,(0,0),fx=0.5,fy=0.5)

imFilename = "bai.jpg"
im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
im = cv2.resize(im,(0,0),fx=0.5,fy=0.5)

imMatches, imReg, h = alignImages_SIFT(im, imReference)
# Print estimated homography
print("Estimated homography : \n", h)


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'match'
> Overload resolution failed:
>  - 'k' is an invalid keyword argument for DescriptorMatcher.match()
>  - 'k' is an invalid keyword argument for DescriptorMatcher.match()
>  - DescriptorMatcher.match() takes at most 2 arguments (3 given)
>  - DescriptorMatcher.match() takes at most 2 arguments (3 given)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 20))
fig2, axes2 = plt.subplots(1, 1, figsize=(10, 20))
fig3, axes3 = plt.subplots(1, 1, figsize=(10, 10))

for i in range(2):
        axes[i].axis('off')
axes2.axis('off')
axes3.axis('off')

axes[0].imshow(imReference[:,:,::-1])
axes[0].set_title('Reference', fontsize=20)

axes[1].imshow(im[:,:,::-1])
axes[1].set_title('Original Img', fontsize=20)

axes2.imshow(imMatches[:,:,::-1])
axes2.set_title('Top matches', fontsize=20)

axes3.imshow(imReg[:,:,::-1])
axes3.set_title('Aligned Img', fontsize=20)

plt.show()

##### 